In [5]:
import numpy as np
import pandas as pd

pd.set_option('display.max_colwidth', None)

file = "newANDfail.csv"
df = pd.read_csv(file)
# df.head()

for index, row in df.iterrows():

    real_output = eval(str(row['real_output']))
    df.at[index, 'Real Output'] = real_output['n200']

    gold_output = eval(str(row['gold_output']))
    df.at[index, 'Gold Output'] = gold_output['n200']
    
    pv = str(row['MC']).split(',')
    df.at[index, 'Process Variation'] = pv[1]

    df.at[index, 'Equal?'] = (gold_output['n200'] == real_output['n200'])

drop_col = ['MC', 'real_output', 'gold_output']
df = df.drop(drop_col, axis=1)

df = df[df['Equal?'] == False]
# df.head(10)

variations = df['Process Variation'].unique()


C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\1792042282.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0010000010000010100001010110' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Real Output'] = real_output['n200']
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\1792042282.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0001000001000001010000101011' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Gold Output'] = gold_output['n200']
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\1792042282.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.01' has dtype incompatible with float64, please explicitly cast to a compatible dt

In [6]:
from functions_file import *

pd.set_option('display.max_columns', None)
fault_distribution = pd.DataFrame()

for pv in variations:

    sample = df[df['Process Variation'] == pv]


    for index, row in sample.iterrows():
        realOutput = str(row['Real Output'])
        goldOutput = str(row['Gold Output'])
        inputPattern = str(row['pattern']).replace(",", "")

        sample.loc[index, 'SA0'] = 1 if O_SA0(realOutput) else 0
        sample.at[index, 'SA1'] = 1 if O_SA1(realOutput) else 0
        sample.at[index, 'EscapeRight'] = 1 if outputEscapeRight(goldOutput, realOutput) else 0
        sample.at[index, 'EscapeLeft'] = 1 if outputEscapeLeft(goldOutput, realOutput) else 0
        sample.at[index, 'Overflow'] = 1 if overflow(realOutput) else 0
        sample.at[index, 'AND->OR'] = 1 if AND_OR(inputPattern, realOutput) else 0
        sample.at[index, 'OR->AND'] = 1 if OR_AND(inputPattern, realOutput) else 0
        sample.at[index, 'alwaysFail_00'] = 1 if alwaysFail_00(inputPattern, realOutput, goldOutput) else 0
        sample.at[index, 'alwaysFail_01'] = 1 if alwaysFail_01(inputPattern, realOutput, goldOutput) else 0
        sample.at[index, 'alwaysFail_10'] = 1 if alwaysFail_10(inputPattern, realOutput, goldOutput) else 0
        sample.at[index, 'alwaysFail_11'] = 1 if alwaysFail_11(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'oftenFail_00'] = 1 if oftenFail_00(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'oftenFail_01'] = 1 if oftenFail_01(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'oftenFail_10'] = 1 if oftenFail_10(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'oftenFail_11'] = 1 if oftenFail_11(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'neverFail_00'] = 1 if neverFail_00(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'neverFail_01'] = 1 if neverFail_01(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'neverFail_10'] = 1 if neverFail_10(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'neverFail_11'] = 1 if neverFail_11(inputPattern, realOutput, goldOutput) else 0

        for sequence in sequences:
            if alwaysFail_sequence(inputPattern, realOutput, goldOutput, sequence) is True:
                sample.at[index, "alwaysFail_" + str(sequence)] = 1
            else:
                sample.at[index, "alwaysFail_" + str(sequence)] = 0
        

    sample['totalTemplates'] = sample.iloc[:, 6:sample.shape[1]-1].sum(axis=1)

    new_row = {}
    for col in sample.columns:
        if pd.api.types.is_numeric_dtype(sample[col]):
            new_row[col] = sample[col].sum()
        else:
            new_row[col] = sample[col].iloc[-1]

    
    sample = pd.concat([sample, pd.DataFrame([new_row], columns=sample.columns)], ignore_index=True)

    print("Process Variation: ", pv)
    display(sample.head())
    display(sample.tail())

    fault_distribution = pd.concat([fault_distribution, pd.DataFrame([sample.iloc[-1]], columns=sample.columns)], ignore_index=True)

C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.loc[index, 'SA0'] = 1 if O_SA0(realOutput) else 0
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.at[index, 'SA1'] = 1 if O_SA1(realOutput) else 0
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

Process Variation:  -0.01


C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['totalTemplates'] = sample.iloc[:, 6:sample.shape[1]-1].sum(axis=1)


,config,pattern,Real Output,Gold Output,Process Variation,Equal?,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111,totalTemplates
0,and_spb_0_and_spb_0_and_spa_0_and,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0010000010000010100001010110,0001000001000001010000101011,-0.01,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,and_spb_0_and_spb_0_and_spa_0_and,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0010000010000010100001010110,0001000001000001010000101011,-0.01,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,and_spb_0_and_spb_0_and_spa_0_and,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0010000010000010100001010110,0001000001000001010000101011,-0.01,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,and_spb_0_and_spb_0_and_spa_0_and,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0010000010000010100001010110,0001000001000001010000101011,-0.01,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,and_spb_0_and_spb_0_and_spa_0_and,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0010000010000010100001010110,0001000001000001010000101011,-0.01,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


,config,pattern,Real Output,Gold Output,Process Variation,Equal?,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111,totalTemplates
1171,xor_spb_0_xor_spb_0_and_spa_0_xor,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0010000010000010100001010110,0001000001000001010000101011,-0.01,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1172,xor_spb_0_xor_spb_0_and_spa_0_xor,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0010000010000010100001010110,0001000001000001010000101011,-0.01,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1173,xor_spb_0_xor_spb_0_and_spa_0_xor,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0010000010000010100001010110,0001000001000001010000101011,-0.01,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1174,xor_spb_0_xor_spb_0_and_spa_0_xor,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0000000000000000000000000000,0001000001000001010000101011,-0.01,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1175,xor_spb_0_xor_spb_0_and_spa_0_xor,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0000000000000000000000000000,0001000001000001010000101011,-0.01,False,40.0,0.0,0.0,1051.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1136.0


...

C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.loc[index, 'SA0'] = 1 if O_SA0(realOutput) else 0
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.at[index, 'SA1'] = 1 if O_SA1(realOutput) else 0
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

Process Variation:  0.01


C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_12156\3289478364.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['totalTemplates'] = sample.iloc[:, 6:sample.shape[1]-1].sum(axis=1)


,config,pattern,Real Output,Gold Output,Process Variation,Equal?,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111,totalTemplates
0,and_spb_0_and_spb_0_and_spa_0_and,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0010000010000010100001010110,0001000001000001010000101011,0.01,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,and_spb_0_and_spb_0_and_spa_0_and,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0010000010000010100001010110,0001000001000001010000101011,0.01,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,and_spb_0_and_spb_0_and_spa_0_and,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0010000010000010100001010110,0001000001000001010000101011,0.01,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,and_spb_0_and_spb_0_and_spa_0_and,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0010000010000010100001010110,0001000001000001010000101011,0.01,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,and_spb_0_and_spb_0_and_spa_0_and,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0010000010000010100001010110,0001000001000001010000101011,0.01,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


,config,pattern,Real Output,Gold Output,Process Variation,Equal?,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111,totalTemplates
836,xor_spb_0_xor_spb_0_and_spa_0_xor,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0010000010000010100001010110,0001000001000001010000101011,0.01,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
837,xor_spb_0_xor_spb_0_and_spa_0_xor,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0010000010000010100001010110,0001000001000001010000101011,0.01,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
838,xor_spb_0_xor_spb_0_and_spa_0_xor,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0000000000000000000000000000,0001000001000001010000101011,0.01,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
839,xor_spb_0_xor_spb_0_and_spa_0_xor,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0000000000000000000000000000,0001000001000001010000101011,0.01,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
840,xor_spb_0_xor_spb_0_and_spa_0_xor,"00,01,10,11,00,01,00,10,00,11,01,01,01,10,01,11,10,11,10,01,10,10,11,10,11,01,11,11",0000000000000000000000000000,0001000001000001010000101011,0.01,False,90.0,0.0,0.0,750.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,930.0


In [7]:
drop_cols = ['config', 'pattern', 'Real Output', 'Gold Output', 'Equal?']
fault_distribution = fault_distribution.drop(drop_cols, axis=1).sort_values(by='Process Variation')
fault_distribution

,Process Variation,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111,totalTemplates
9,-0.0,0.0,0.0,0.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0
0,-0.01,40.0,0.0,0.0,1051.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1136.0
7,-0.02,0.0,0.0,0.0,1125.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1130.0
16,-0.03,0.0,0.0,0.0,733.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,733.0
10,-0.04,45.0,0.0,0.0,1870.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1965.0
1,-0.05,65.0,0.0,0.0,1249.0,0.0,0.0,0.0,0.0,0.0,0.0,95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,1419.0
2,-0.06,95.0,0.0,0.0,841.0,0.0,0.0,0.0,0.0,0.0,0.0,105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1041.0
15,-0.07,269.0,0.0,0.0,1125.0,0.0,0.0,0.0,0.0,0.0,0.0,355.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,5.0,1764.0
11,-0.08,0.0,0.0,0.0,1735.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1740.0
3,-0.09,45.0,0.0,0.0,510.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,630.0


In [8]:
fault_distribution_ratio = fault_distribution.iloc[:, 1:-1].div(fault_distribution['totalTemplates'], axis=0)
fault_distribution_ratio.insert(0, 'Process Variation', fault_distribution['Process Variation'])
fault_distribution_ratio


,Process Variation,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111
9,-0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
0,-0.01,0.035211,0.0,0.0,0.925176,0.0,0.0,0.0,0.0,0.0,0.0,0.039613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
7,-0.02,0.000000,0.0,0.0,0.995575,0.0,0.0,0.0,0.0,0.0,0.0,0.004425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.004425
16,-0.03,0.000000,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
10,-0.04,0.022901,0.0,0.0,0.951654,0.0,0.0,0.0,0.0,0.0,0.0,0.025445,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,-0.05,0.045807,0.0,0.0,0.880197,0.0,0.0,0.0,0.0,0.0,0.0,0.066949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.007047,0.0,0.0,0.0,0.000000
2,-0.06,0.091258,0.0,0.0,0.807877,0.0,0.0,0.0,0.0,0.0,0.0,0.100865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
15,-0.07,0.152494,0.0,0.0,0.637755,0.0,0.0,0.0,0.0,0.0,0.0,0.201247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000567,0.0,0.0,0.0,0.007937,0.0,0.0,0.0,0.002834
11,-0.08,0.000000,0.0,0.0,0.997126,0.0,0.0,0.0,0.0,0.0,0.0,0.002874,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.002874
3,-0.09,0.071429,0.0,0.0,0.809524,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.007937,0.0,0.0,0.0,0.007937
